# This Jupyter Notebook calculates an accurate and an approximate gauss filter

In [1]:
from PIL import Image
import statistics as st
import random
import numpy as np
import time
import cv2
from PIL import Image
from matplotlib.pyplot import imshow
from skimage import metrics as ms

%matplotlib inline

In [2]:
def gauss(s, M, r):
    
    im = Image.open(s)
    width = im.size[0]
    height = im.size[1]
    pix = im.load()
    new_pix = []
    sm_r = 0
    sm_g = 0
    sm_b = 0
    g = []
    
    #Ignoring a border of 5 pixels
    if r == "accurate":
        for h in range(height-5):
            for w in range(width-5):
                for i in range(5):
                    for k in range(5):
                        b = tuple([q*M[i,k] for q in pix[w+k, h+i]])      #Multiply Matrix with tupel of rgb values
                        sm_r += b[0]
                        sm_g += b[1]
                        sm_b += b[2]
                new_pix.append(tuple([int(np.floor(sm_r/256)), int(np.floor(sm_g/256)), int(np.floor(sm_b/256))]))
                sm_r = 0
                sm_g = 0
                sm_b = 0
        
        d = 0
        for h in range(height-5):
            for w in range(width-5):
                pix[w+2, h+2] = new_pix[d]
                d += 1
        im.save('Eddy_noise_accurate_gauss.jpg')
        
    #Ignoring border
    if r == "approximate":
        hc = 0
        for h in range(int((height-5)/3)-3):
            wc = 0
            for w in range(int((width-6)/3)):
                for i in range(5):
                    for k in range(5):
                        b = tuple([q*M[i,k] for q in pix[wc+k, hc+i]])
                        sm_r += b[0]
                        sm_g += b[1]
                        sm_b += b[2]
                new_pix.append(tuple([int(np.floor(sm_r/256)), int(np.floor(sm_g/256)), int(np.floor(sm_b/256))]))
                sm_r = 0
                sm_g = 0
                sm_b = 0
                wc += 3
            hc += 3
            
        d = 0
        hc = 0
        
        #Give 3 pixels to the right and three pixels down the same value
        #E.g. pix[0,0] = pix[0, 1] = pix[0, 3] = pix[1, 0] = ... = pix[3, 3] = new_pix[0]
        for h in range(int((height-5)/3)-3):
            wc = 0
            for w in range(int((width-6)/3)):
                pix[wc+2, hc+2] = new_pix[d]
                pix[wc+3, hc+2] = new_pix[d]
                pix[wc+4, hc+2] = new_pix[d]
                
                pix[wc+2, hc+3] = new_pix[d]
                pix[wc+3, hc+3] = new_pix[d]
                pix[wc+4, hc+3] = new_pix[d]
                
                pix[wc+2, hc+4] = new_pix[d]
                pix[wc+3, hc+4] = new_pix[d]
                pix[wc+4, hc+4] = new_pix[d]
                d += 1
                wc += 3
            hc += 3
        im.save('Eddy_noise_approximate_gauss.jpg')

In [3]:
#Calculate the PSNR
def calc_psnr(org, con):
    img1 = cv2.imread(org)
    img2 = cv2.imread(con)
    psnr = cv2.PSNR(img1, img2)
    
    return psnr

In [4]:
#Calculate SSIM
def calc_ssim(org, con):
    img1 = cv2.imread(org)
    img2 = cv2.imread(con)
    ssim = ms.structural_similarity(img1, img2, multichannel=True)
    
    return ssim

In [5]:
M = np.matrix('1 4 6 4 1; 4 16 24 16 4; 6 24 36 24 6; 4 16 24 16 4; 1 4 6 4 1')
s1 = time.time()
gauss('Eddy_noise.bmp', M, 'accurate')
e1 = time.time()

s2 = time.time()
gauss('Eddy_noise.bmp', M, 'approximate')
e2 = time.time()

print("Time it took for accurate gauss filter:", e1-s1)
print("Time it took for approximate gauss filter:", e2-s2, "\n")

print("PSNR for the accurate gauss filter:", calc_psnr('Eddy.bmp', 'Eddy_noise_accurate_gauss.jpg'))
print("PSNR for the approximate gauss filter:", calc_psnr('Eddy.bmp', 'Eddy_noise_approximate_gauss.jpg'), "\n")

print("SSIM for the accurate gauss filter:", calc_ssim('Eddy.bmp', 'Eddy_noise_accurate_gauss.jpg'))
print("SSIM for the approximate gauss filter:", calc_ssim('Eddy.bmp', 'Eddy_noise_approximate_gauss.jpg'), "\n")

Time it took for accurate gauss filter: 364.73216247558594
Time it took for approximate gauss filter: 32.64933753013611 

PSNR for the accurate gauss filter: 23.074370246338745
PSNR for the approximate gauss filter: 22.615157166741927 

SSIM for the accurate gauss filter: 0.5361530390297052
SSIM for the approximate gauss filter: 0.531503882071111 

